In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import SimpleITK as sitk

from src.evaluation.scores import dice

In [2]:
prediction_folder = '../data/segmentation_output_renamed'
groundtruth_folder = '../data/hecktor_nii/'
bb_filepath = '../data/bbox.csv'

In [11]:
# List of the files in the validation
prediction_files = [
            f
            for f in Path(prediction_folder).rglob('*.nii.gz')
        ]

# The list is sorted, so it will match the list of ground truth files
prediction_files.sort(key=lambda x: x.name.split('_')[0])   
 
# List of the patient_id in the validation
patient_name_predictions = [
    f.name.split('.')[0][:7] for f in prediction_files
]


In [13]:
# List of the ground truth files
groundtruth_files = [
    f for f in Path(groundtruth_folder).rglob('*gtvt.nii.gz') if f.name.split('_')[0] in patient_name_predictions
]

In [15]:
# The bounding boxes will be used to compute the Dice score within.
bb_df = pd.read_csv(bb_filepath).set_index('PatientID')

In [19]:
# DataFrame to store the results
results_df = pd.DataFrame(columns=['PatientID', 'Dice Score'])

resampler = sitk.ResampleImageFilter()
resampler.SetInterpolator(sitk.sitkNearestNeighbor)

for f in prediction_files:
    patient_name = f.name.split('.')[0][:7]
    gt_file = [k for k in groundtruth_files if k.name[:7] == patient_name][0]

    print('Evaluating patient {}'.format(patient_name))

    sitk_pred = sitk.ReadImage(str(f.resolve()))
    sitk_gt = sitk.ReadImage(str(gt_file.resolve()))
    resampling_spacing = np.array(sitk_gt.GetSpacing())

    bb = np.array([
        bb_df.loc[patient_name, 'x1', ], bb_df.loc[patient_name, 'y1', ],
        bb_df.loc[patient_name, 'z1', ], bb_df.loc[patient_name, 'x2', ],
        bb_df.loc[patient_name, 'y2', ], bb_df.loc[patient_name, 'z2', ]
    ])

    image_size = np.round((bb[3:] - bb[:3]) / resampling_spacing).astype(int)
    resampler.SetOutputOrigin(bb[:3])
    resampler.SetSize([int(k) for k in image_size])
    resampler.SetReferenceImage(sitk_gt)

    sitk_gt = resampler.Execute(sitk_gt)
    sitk_pred = resampler.Execute(sitk_pred)

    # Store the results
    results_df = results_df.append(
        {
            'PatientID':
            patient_name,
            'Dice Score':
            dice(sitk.GetArrayFromImage(sitk_gt),
                 sitk.GetArrayFromImage(sitk_pred)),
        },
        ignore_index=True)


Evaluating patient CHGJ007
Evaluating patient CHGJ008
Evaluating patient CHGJ010
Evaluating patient CHGJ013
Evaluating patient CHGJ015
Evaluating patient CHGJ016
Evaluating patient CHGJ017
Evaluating patient CHGJ018
Evaluating patient CHGJ025
Evaluating patient CHGJ026
Evaluating patient CHGJ028
Evaluating patient CHGJ029
Evaluating patient CHGJ030
Evaluating patient CHGJ031
Evaluating patient CHGJ032
Evaluating patient CHGJ034
Evaluating patient CHGJ035
Evaluating patient CHGJ036
Evaluating patient CHGJ037
Evaluating patient CHGJ038
Evaluating patient CHGJ039
Evaluating patient CHGJ043
Evaluating patient CHGJ046
Evaluating patient CHGJ048
Evaluating patient CHGJ050
Evaluating patient CHGJ052
Evaluating patient CHGJ053
Evaluating patient CHGJ055
Evaluating patient CHGJ057
Evaluating patient CHGJ058
Evaluating patient CHGJ062
Evaluating patient CHGJ065
Evaluating patient CHGJ066
Evaluating patient CHGJ067
Evaluating patient CHGJ069
Evaluating patient CHGJ070
Evaluating patient CHGJ071
E

In [20]:
results_df['Dice Score'].values

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1.])